# Collections

## Prerequisite: What are containers in Python?
* Object that contains 0+ other objects
* Examples: `tuple`, `list`, `set`, `dict`
* Abstract base class is `collections.abc.Container` (`collections.Container` in Py2)
    * Supports the `__contains__` method which allows for the use of the `in` keyword (`x in y`)

## What are collections in Python?
* Specialized containers as alternative to built-ins
* Examples: `namedtuple`, `Counter`, `OrderedDict`



---
## `namedtuple`
* Factory function that generates an indexable, iterable class (subclass of `tuple`) with the named attribute fields
* Why use it? 
    * Lightweight
    * **Require no more memory than a tuple containing identical objects**

In [1]:
##### COMPARING MEMORY AND ATTRIBUTES FOR NAMEDTUPLE GENERATED CLASS, TUPLE, AND MANUAL CLASS DEFINITION #####
from collections import namedtuple
import sys
# from guppy import hpy; h=hpy()

# TUPLE 
p0 = (8, 2, 5)
print("tuple class".upper())
print(f"__repr__: p0 = {p0}")
print(f"indexing: p0[1] = {p0[1]} (p0.y does not exist)")
print(f"__dict__ attr: {hasattr(p0, '__dict__')}")
print(f"class sizeof: {sys.getsizeof(tuple)} bytes")
print(f"instance sizeof: {sys.getsizeof(p0)} bytes")
# print(f"class sizeof: {h.iso(tuple).size} bytes")
# print(f"instance sizeof: {h.iso(p0).size} bytes")

# NAMEDTUPLE (FACTORY FUNCTION THAT RETURNS A CLASS)
Point2D = namedtuple("Point2D", ["x", "y"]) # or namedtuple("Point", "x y") or namedtuple("Point", "x, y")
Point3D = namedtuple("Point3D", Point2D._fields + tuple("z"))
p1 = Point3D(p0[0], y=p0[1], z=p0[2])

print("\nclass generated from namedtuple".upper())
print(f"__repr__: p1 = {p1}")
print(f"indexing: p1.y = {p1.y} == p1[1] = {p1[1]}")
print(f"__dict__ attr: {hasattr(p1, '__dict__')}")
print(f"class sizeof: {sys.getsizeof(Point3D)} bytes")
print(f"instance sizeof: {sys.getsizeof(p1)} bytes")
# print(f"class sizeof: {h.iso(Point3D).size} bytes")
# print(f"instance sizeof: {h.iso(p1).size} bytes")

# MANUAL CLASS
class Point3D(tuple):
    def __init__(self, it):
        if len(it) > 3:
            raise ValueError(f"3 values expected in iterable, {len(it)} received")
        else:
            self.x = it[0]
            self.y = it[1]
            self.z = it[2]
     
p2 = Point3D(p0)
print("\nmanual class definition".upper())
print(f"__repr__: p2 = {p2}")
print(f"indexing: p2.y = {p2.y} == p2[1] = {p2[1]}")
print(f"__dict__ attribute: {hasattr(p2, '__dict__')}")
print(f"class sizeof: {sys.getsizeof(Point3D)} bytes")
print(f"instance sizeof: {sys.getsizeof(p2)} bytes")
# print(f"class sizeof: {h.iso(Point3D).size} bytes")
# print(f"instance sizeof: {h.iso(p2).size} bytes")

TUPLE CLASS
__repr__: p0 = (8, 2, 5)
indexing: p0[1] = 2 (p0.y does not exist)
__dict__ attr: False
class sizeof: 400 bytes
instance sizeof: 80 bytes

CLASS GENERATED FROM NAMEDTUPLE
__repr__: p1 = Point3D(x=8, y=2, z=5)
indexing: p1.y = 2 == p1[1] = 2
__dict__ attr: False
class sizeof: 896 bytes
instance sizeof: 80 bytes

MANUAL CLASS DEFINITION
__repr__: p2 = (8, 2, 5)
indexing: p2.y = 2 == p2[1] = 2
__dict__ attribute: True
class sizeof: 1064 bytes
instance sizeof: 88 bytes


In [2]:
##### MAPPING ITEMS IN A CSV FILE TO A NAMED TUPLE CLASS ##### 
from collections import namedtuple
import csv

# import urllib.request
# url = "https://gist.githubusercontent.com/GoodmanSciences/c2dd862cd38f21b0ad36b8f96b4bf1ee/raw/1d92663004489a5b6926e944c1b3d9ec5c40900e/Periodic%2520Table%2520of%2520Elements.csv"
# with urllib.request.urlopen(url) as response:
#    data = response.read()

Element = namedtuple("Element", "AtomicNumber,Element,Symbol,AtomicMass,NumberofNeutrons,NumberofProtons,NumberofElectrons,Period,Group,Phase")
with open("./Periodic Table of Elements.csv") as f:
    for el in map(Element._make, csv.reader(f)): 
        print(el)


Element(AtomicNumber='1', Element='Hydrogen', Symbol='H', AtomicMass='1.007', NumberofNeutrons='0', NumberofProtons='1', NumberofElectrons='1', Period='1', Group='1', Phase='gas')
Element(AtomicNumber='2', Element='Helium', Symbol='He', AtomicMass='4.002', NumberofNeutrons='2', NumberofProtons='2', NumberofElectrons='2', Period='1', Group='18', Phase='gas')
Element(AtomicNumber='3', Element='Lithium', Symbol='Li', AtomicMass='6.941', NumberofNeutrons='4', NumberofProtons='3', NumberofElectrons='3', Period='2', Group='1', Phase='solid')
Element(AtomicNumber='4', Element='Beryllium', Symbol='Be', AtomicMass='9.012', NumberofNeutrons='5', NumberofProtons='4', NumberofElectrons='4', Period='2', Group='2', Phase='solid')
Element(AtomicNumber='5', Element='Boron', Symbol='B', AtomicMass='10.811', NumberofNeutrons='6', NumberofProtons='5', NumberofElectrons='5', Period='2', Group='13', Phase='solid')
Element(AtomicNumber='6', Element='Carbon', Symbol='C', AtomicMass='12.011', NumberofNeutrons

---
## `deque`
* "Generalization of stacks and queues (the name is pronounced 'deck' and is short for 'double-ended queue')"
* Implementation is done in C/CPython to keep performance overhead low
* Why use it?
    * **Thread-safe**
    * Memory efficient left and right appends and pops (**O(1) performance from both directions** vs. O(n) for left side pop and append for `list`) 
    * Additional features (methods): `rotate`, `maxlen`, `extendleft`

In [3]:
##### COMPARING TIME OF IDENTICAL RESULT W/ DEQUE AND LIST #####
from collections import deque

def op_to_time1(n=100):
    x = deque()
    for i in range(n):
        x.appendleft(i)
    return x

def op_to_time2(n=100):
    x = []
    for i in range(n):
        x.append(i)
    return list(reversed(x))

def op_to_time3(n=100):
    x = []
    for i in range(n):
        x.insert(0, i)
    return x
        
%timeit op_to_time1() # appendleft is fastest
%timeit op_to_time2()
%timeit op_to_time3()

8.57 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.7 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.2 µs ± 665 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
##### DEQUE AS A FIXED LENGTH QUEUE #####
from collections import deque
from logging import LogRecord
import random, time

escalation = ("CRITICAL", "ERROR", "WARNING", "INFO", "DEBUG")
status = ("connection interrupted", "connection reset", "connection initiated", "improper config")

queue = deque([], maxlen=10)
for i in range(25): # insert 25 times but only 10 are kept in queue
    queue.appendleft(LogRecord('root', random.choice(escalation), "server.py", random.randint(0, 100), f"{time.time()}: {random.choice(status)}", "", None))

print(f"len(queue) = {len(queue)}")
queue

len(queue) = 10


deque([<LogRecord: root, INFO, server.py, 99, "1581625461.2797258: connection reset">,
       <LogRecord: root, ERROR, server.py, 63, "1581625461.279687: improper config">,
       <LogRecord: root, DEBUG, server.py, 94, "1581625461.279479: connection initiated">,
       <LogRecord: root, WARNING, server.py, 9, "1581625461.2794123: connection interrupted">,
       <LogRecord: root, ERROR, server.py, 90, "1581625461.279341: connection interrupted">,
       <LogRecord: root, DEBUG, server.py, 31, "1581625461.2793057: connection initiated">,
       <LogRecord: root, INFO, server.py, 37, "1581625461.279265: improper config">,
       <LogRecord: root, WARNING, server.py, 28, "1581625461.278993: connection interrupted">,
       <LogRecord: root, CRITICAL, server.py, 73, "1581625461.278954: connection reset">,
       <LogRecord: root, INFO, server.py, 88, "1581625461.2789073: improper config">])

In [5]:
##### ADDITIONAL METHODS FOR DEQUE #####
from collections import deque
x = deque(range(10))
x.extendleft(reversed(range(-5, 0))) # deque.extendleft() inserts from arguments right to left
print(x)
x.rotate(-5)
print(x)

deque([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -5, -4, -3, -2, -1])


---
## `ChainMap`
* Groups multiple maps (generically dictionaries in Python)
* What is a map?
    * "[The correspondence of elements in one set to elements in the same set or another set.](https://www.thefreedictionary.com/map)"
    * "[A mapping object maps values of one type (the key type) to arbitrary objects.](https://docs.python.org/2.0/lib/typesmapping.html)"
* Dictionary methods `update`, `clear`, `pop`, `popitem`, (...possibly more) only apply to first dictionary (referred to as parent)
* Why use it?
    * Underlying maps are stored as references (lightweight); however, changes to maps outside of ChainMap propogate
    * Allows dictionary lookups across multiple dictionaries (`__contains__`/`in`, `dict["key"]`)
    * Any others?

In [6]:
##### SHOWING CHAINMAP ONLY HOLDS REFERENCES TO EXTERNAL MAPS #####
from collections import ChainMap

fav_fruits1 = {"apple": "John", "cherry": "Hannah", "orange": "Wendy"}
fav_fruits2 = {"orange": "Tyler", "kiwi": "Jessie", "apple": "Emily"}

fav_fruits = ChainMap(fav_fruits1, fav_fruits2)
print(f"kiwi and cherry in fav_fruits: {'kiwi' in fav_fruits} and {'cherry' in fav_fruits}")
print(f"keys: {list(fav_fruits.keys())}")
fav_fruits1.update({"pineapple":"Katie"}) # not updating fav_fruits directly
print(f"keys: {list(fav_fruits.keys())}")

kiwi and cherry in fav_fruits: True and True
keys: ['orange', 'kiwi', 'apple', 'cherry']
keys: ['orange', 'kiwi', 'apple', 'cherry', 'pineapple']


In [7]:
##### SHOWING CHAINMAP ONLY HOLDS REFERENCES TO EXTERNAL MAPS #####
from collections import ChainMap
import sys

ALPHABET = "abcdefghijklmnopqrstuvwxyz"

cm = ChainMap()
for alpha, i in zip(ALPHABET, range(0, 25)):
    cm.update({i: alpha})
        
d = {}
for alpha, i in zip(ALPHABET, range(0, 25)):
    d.update({i: alpha})

print("ChainMap".upper())
print(f"instance sizeof: {sys.getsizeof(cm)} bytes") # only stores references

print("\ndictionary".upper())
print(f"instance sizeof: {sys.getsizeof(d)} bytes")

CHAINMAP
instance sizeof: 64 bytes

DICTIONARY
instance sizeof: 1192 bytes


---
## `Counter`
* `dict` where keys are object to count and values are the count
* Why use it?
    * Support addition, subtraction, intersection, and union operators/methods
    * Additonal methods `most_common`, `elements`

In [8]:
##### DEMONSTRATE COUNTER FROM ITERABLE ######
from collections import Counter

c = Counter("happy-go-lucky") # alternate c1 = Counter(a=8, b=2, c=14)
print(c)
print(f"c['t'] = {c['t']}")

Counter({'p': 2, 'y': 2, '-': 2, 'h': 1, 'a': 1, 'g': 1, 'o': 1, 'l': 1, 'u': 1, 'c': 1, 'k': 1})
c['t'] = 0


In [9]:
from collections import Counter

c1 = Counter()
with open('words', 'rt') as f: # words copied from /usr/share/dict/words
    for line in f:
        c1.update(line.rstrip().lower())
        
print(c1.most_common(5))

c2 = Counter(e = c1["e"])
(c1 - c2).most_common(5)

[('e', 235331), ('i', 201032), ('a', 199554), ('o', 170692), ('r', 160985)]


[('i', 201032), ('a', 199554), ('o', 170692), ('r', 160985), ('n', 158743)]

In [10]:
c = Counter(a=2, b=-4)
print(f"-c: {-c}")
print(f"+c: {+c}")

-c: Counter({'b': 4})
+c: Counter({'a': 2})


---
## `OrderedDict`
* A specialized `dict` that maintains order during iteration and has methods for rearranging the order
* Why use it?
    * Iterating over a `dict` type has not be guaranteed to be consistent and repeatable in previous Python versions (although since 3.6 the `dict` maintains insertion order and in 3.7 it became language spec)
    * Allows additional methods to be called on it: `move_to_end`, `popitem`, and `__reversed__` (3.8 added this for `dict`)

### Sample problem: Raindrops (exercism.io)
* [My solution](https://exercism.io/tracks/python/exercises/raindrops/solutions/6b156cb11882422db8f57520028f5c7b) shown below
* [Exercism exercise on GitHub](https://github.com/exercism/python/tree/master/exercises/raindrops)

In [11]:
##### UTILIZING THE ORDERED ITERATION OVER AN ORDEREDDICT #####
from collections import OrderedDict

def raindrops(num):
    snd_dict = OrderedDict(((3, "Pling"), (5, "Plang"), (7, "Plong")))
    snd = [snd_dict[k] for k in snd_dict if num % k == 0] # keeps order during iteration b/c it's an OrderedDict!
    if len(snd) == 0:
        return str(num)
    return "".join(snd)

print(raindrops(105)) # "PlingPlangPlong"
print(raindrops(8)) # "8"

PlingPlangPlong
8


In [12]:
##### ADDITIONAL METHODS ON ORDEREDDICT #####
from collections import OrderedDict

x = OrderedDict({"Tim": 4, "Johnny": 8, "Sabrina": 3, "Jackson": 6})
x.move_to_end("Johnny")
print(x)
x.move_to_end("Sabrina", last=False)
print(x)
for pair in reversed(x.items()):
    print(pair)

OrderedDict([('Tim', 4), ('Sabrina', 3), ('Jackson', 6), ('Johnny', 8)])
OrderedDict([('Sabrina', 3), ('Tim', 4), ('Jackson', 6), ('Johnny', 8)])
('Johnny', 8)
('Jackson', 6)
('Tim', 4)
('Sabrina', 3)


---
## `defaultdict`
* Allows defining a subtype to be stored in a `dict`
* When a key is first encountered without a value, the value takes the "zero" value for that type (explain)
* Why use it?
    * Default value generation for a key can be handy in that it allows you to utilize the methods of that type immediately
    * Any others?

In [13]:
##### EXPLAINING DEFAULT TYPES AND RESPECTIVE ZERO VALUES #####
from collections import defaultdict
d = defaultdict(int) # toggle
# d = defaultdict(list)
# d = defaultdict(str)
d["MyInitialKey"]
print(d)

defaultdict(<class 'int'>, {'MyInitialKey': 0})


In [14]:
##### LEVERAGING THE DEFAULTDICT DEFAULT TYPE #####
from collections import defaultdict

s1 = 'mississippi'
d1 = defaultdict(int)
for k in s1:
    d1[k] += 1
print(d1)
    
s2 = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d2 = defaultdict(list)
for k, v in s2:
    d2[k].append(v)
print(d2)

defaultdict(<class 'int'>, {'m': 1, 'i': 4, 's': 4, 'p': 2})
defaultdict(<class 'list'>, {'yellow': [1, 3], 'blue': [2, 4], 'red': [1]})


---
## `UserDict`, `UserList`, and `UserString`
* Wrappers around `dict`, `list`, and `string`, respectively
* Why use it?
    * Allow for more convenient subclassing of these built-ins as they provide additional attributes

In [15]:
##### ADDITIONAL ATTRIBUTE `data` #####
from collections import UserList
class MyList(UserList):
    pass
    
x = MyList([0]*5)
print(f"x.data: {x.data}")

x.data: [0, 0, 0, 0, 0]


---
## ABCs (Abstract Base Classes)
* Factory classes intended to be subclassed, but unable to be instantiated themselves
* Why use them?
    * Define API for subclasses
    * Ex: Blueprints, plugins where 3rd party implements subclasses
    * Allows for creating more generic implementations using `issubclass` and `isinstance`

In [16]:
import abc
import math

class Shape(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def __init__(self):
        pass
        
    @abc.abstractmethod # indicates area must be implemented in subclass -- Python enforces this
    def area(self):
        pass
    
class Rectangle(Shape):
    l = 0
    w = 0
        
    def __init__(self, l, w):
        self.l, self.w = l, w
        
    def area(self):
        return self.l*self.w

class Ellipse(Shape):
    r1 = 0
    r2 = 0
        
    def __init__(self, r1, r2):
        self.r1, self.r2 = r1, r2
        
    def area(self): # show NotImplementedError by commenting this method out
        return self.r1*self.r2*math.pi

e = Ellipse(1, 2)

In [17]:
from collections.abc import Iterable

# generic function that only needs to accept iterable objects -- can be string, list, tuple
def length(x):
    if not isinstance(x, Iterable): # checks whether x is an instance of the class or subclass of Iterable
        raise ValueError(f"type {type(x)} is not iterable")
    c = 0
    for val in x:
        c += 1
    return c

print(length([3, 5, 7]))
print(length("howdy"))
# print(counter(3)) # will fail b/c not an Iterable subclass (doesn't contain __iter__ method)

3
5


---
## Repo
This notebook can be found at https://gitlab.com/estysdesu/explorecollections or the GitHub mirror (same URL except github.com)

## Questions